# CH6 실제 시스템과의 조합
- 추천 알고리즘을 실제 서비스에 적용하기 위해서는 DB, API를 포함한 설계가 필요
- 이번 장에서는 아래 항목에 대한 시스템 구축에 필요한 설계 및 패턴을 소개
  - 배치/실시간 추천
  - 추천 시스템 설계 패턴
  - 다단계 추천이나 근사 최근접 탐색을 통한 노력
  - 로그 설계
  - 뉴스 서비스에서의 실제 사례

## 6.1 시스템 개요
### 6.1.1 배치 추천과 실시간 추천
- 이번 장에서 배치/실시간 추천을 아래와 같이 정의
  - 배치 추천 : 모델 학습, 피처 추출 및 업데이트, 예측 모두 일괄 수행
  - 실시간 추천 : 사용자 클릭 등을 트리거로 피처 추출 및 업데이트, 예측(사용자 요청시)을 실시간으로 수행
- 배치 : 아이템이나 사용자 신규 추가 및 업데이트 빈도가 적고 업데이트 요구 수준이 비교적 낮을 때 적합
- 실시간 : 아이템이나 사용자 신규 추가 및 업데이트 빈도가 많고 업데이트 요구 수준이 높을 때 적합

### 6.1.2 대표적인 추천 시스템 개요
#### 개요 추천
- 신규순, 인기순으로 아이템을 표시하는 것

#### 연관 아이템 추천
- 사전에 유사도를 계산해 비슷한 아이템군을 DB에 저장해 결과를 반환
- item_id를 db에 보내고 계산 결과를 반환
- A/B 테스트로 실험하며 알고리즘을 개선할 수 있음

#### 개인화 추천
- 각 사용자별로 추천하는 아이템을 미리 계산하여 db에 저장
- 개인화 알고리즘 결과를 별도로 유지하고 개인화 결과를 조합해 사용자에게 제시
- 벡터 기반 개인화는 최근 많이 활용됨
  - 아이템과 사용자의 특징을 벡터화해서 db에 저장
- item2vec, 딥러닝 모델 적용, 아이템 벡터를 시계열에 따라 RNN 모델에서 벡터화 하는 등의 방법이 있음

### 6.1.3 다단계 추천
- 트래픽이 많을 경우 시스템 부하를 고려해 설계해야 함
- 다단계 추천 : 시스템 부하를 낮게 유지하며 정밀도가 높은 추천을 수행하기 위해 '후보 선택', '스코어링', '재순위' 같은 다단계로 처리를 구분하는 방법
- 2단계 추천(two-stage recommendation) : 후보 선택, 스코어링/재순위를 나눠 부르는 말
  - 후보 선택(10,000~아이템)->스코어링(100~10,000아이템)->재순위(10~100아이템)

#### 후보 선택
- 막대한 아이템으로부터 추천 후보가 될 아이템을 추출
  - 수십억개에 이르는 추천 아이템 후보를 100~10,000개로 줄임
  - 후보 선택 로직을 조합해 다양한 관점에서 후보 아이템을 추출 가능

#### 스코어링
- 실제 사용자에게 제시할 아이템에 점수를 부여
- 부하가 높은 모델 특히 머신러닝 모델에 의한 높은 정확도의 추론을 활용

#### 재순위
- 스코어링에서 선택된 아이템을 나열하도록 처리
  - 순위 정체의 균형을 고려해 비슷한 아이템만 나열되지 않도록 하거나 아이템 밀집도를 고려해 나열하도록 처리
  - 다양성, 밀집도, 콘텐츠 특유의 정보를 활용해 사용자 경험이나 비즈니스 로직이 고려된 추천을 실현하기 위해 수행됨

### 6.1.4 근사 최근접 탐색
- 추천 알고리즘들은 공통적으로 사용자와 아이템을 벡터로 표현하고 벡터의 유사도를 기반으로 추천을 수행
- 최근접 탐색에서 벡터의 유사도를 단순히 계산하면 아이템이 최대가 되는 경우 응답에 시간이 걸려 만족도가 저하될 수 있음
- 실무에서 이용할 때의 주의점을 알아보면서 근사 최근접 탐색에 대해 살펴본다
- 근사 최근접 탐색을 사용한 추천은 주로 아래 2가지 단계로 이루어짐
  1. 아이템(사용자)의 벡터에 인덱스를 붙인다.
  2. 해당 인덱스를 사용해 사용자에게 추천 아이템을 추천한다.

![벡터를 영역별로 분할해 인덱스화한다](./images/img_6-10.png)

#### 아이템과 사용자 벡터 모두를 일 단위로 생성하는 경우
- 행렬 분해, item2vec으로 아이템과 사용자 벡터를 매일 새롭게 생성하는 경우
- 주의점: 알고리즘을 재학습시키면 어제 계산된 벡터와 오늘 계산된 벡터의 차원의 의미가 달라짐
  - 따라서 아이템과 사용자 벡터 모두를 일 단위로 새롭게 생성하는 경우 근사 최근접 탐색에서 인덱스도 다음과 같이 일 단위로 붙여야 함
    1. 아이템과 사용자 벡터를 생성한다
    2. 아이템 벡터에 인덱스를 붙인다
    3. 인덱스를 사용해 사용자에게 추천 아이템을 추천한다

#### 사용자 벡터만 일 단위로 생성하는 경우
- 계산 속도나 서버 비용 관점에서 한계가 있을 때 사용자 벡터만 일 단위로 업데이트하는 방법을 생각할 수 있음
- 아래와 같이 활용
- **주 단위로 다음을 처리**
  1. 아이템 벡터를 생성
  2. 아이템 벡터에 인덱스를 붙임
- **일 단위로 다음을 처리**
  1. 사용자 벡터를 생성
  2. 인덱스를 사용해 사용자에게 추천 아이템을 추천

- 일 단위 계산에서는 사용자 벡터 생성과 추천 계산만 실행하므로 시스템 부하가 낮아질 것이라 기대
- 아이템 벡터는 주 1회만 생성하므로 새로운 아이템은 다음번 처리 때까지 추천할 수 없다는 것이 문제

#### 모델을 유지하고 일 단위로 벡터를 생성하는 경우
- 새로 사용될 수 있게 된 아이템과 사용자 벡터를 새롭게 추천에 활용
- **가장 먼저 다음을 처리**
  1. 아이템과 사용자 벡터를 생성하고 이떄의 모델을 저장
  2. 아이템 벡터에 인덱스를 붙임
- **일 단위로 다음을 처리**
  1. 저장한 모델을 사용해 아이템과 사용자의 벡터를 생성
  2. 신규 아이템 벡터에 인덱스를 붙임
  3. 그 인덱스를 사용해 사용자에게 추천 아이템을 추천

- 위 방법의 장점은 전체 인덱스를 붙이는 작업이 한번에 완료되며 그 후에는 새로운 아이템의 인덱스만 붙이면 된다는 것
- 하지만 기존의 아이템은 계속 같은 인덱스가 붙으므로 추천 정확도가 떨어질 우려가 있음
- 따라서 추천 시스템에서는 앞의 2가지 방법을 많이 사용하고 마지막 방법은 유사 이미지 검색이나 콘텐츠 기반 추천 시스템에서 많이 사용

- 구체적인 근사 최근접 탐색 방법으로는 LSH, NMSLIB, Faiss, Annoy가 있음
- 아래의 순서도 참고
  ![근사 최근접 참색의 선택 방법 순서도](./images/img_6-11.png)

- 아래는 각 근사 최근접 탐색을 실험한 벤치 마크 결과도
  - 가로 축이 정확성(Recall), 세로 축이 속도(Query per second)
    ![Annoy 제작자가 만든 성능 비교](./images/img_6-12.png)

### 6.2 로그 설계
- 무엇을 남겨야 하는지, 어떻게 활용할지의 관점으로 기술
- 각각 클라이언트 사이드와 서버 사이드에서 얻을 수 있는 로그가 있음

### 6.2.1 클라이언트 사이드 로그
- 사용자가 어떤 아이템을 열람하고 어떤 아이템을 클릭했는지 경과나 결과를 기록하는 것이 주 목적

#### 사용자 행동
- 클라이언트 사이드 로그에서 대표적인 행동 로그

#### 성능
- 사용자가 요청한 페이지가 실제로 완전히 표시될 떄 까지 걸리는 시간 또는 조작이 가능할 때까지 걸린 시간

#### 에러와 크래시
- 웹 브라우저, 애플리케이션에서 발생한 에러나 크래시 로그

### 6.2.2 서버 사이드 로그
#### 성능
- 클라이언트 요청에 응답 시간

#### 시스템 응답
- 클라이언트로부터의 요청 수나 응답 수 또는 정상 응답 비율

#### 시스템 처리 정보
- 캐시의 히트 비율, CPU 부하, 메모리 사용율, 에러나 예외 수 등 시스템 처리 경과나 상황을 기록

### 6.2.3 사용자 행동 
- 일반적으로 서비스에서는 표시 > 클릭 > 구매 순으로 사용자 행동량이 많음

## 6.3 실제 시스템 예
- 배치 추천의 예시로 뉴스 서비스에서 대규모 푸시 알림을 수행
- 실시간 추천의 예시로 뉴스 서비스 기사 추천을 수행

### 6.3.1 배치 추천
![뉴스 서비스의 푸시 알림 과정](./images/img_6-13.png)

- 행동로그를 사용해 클릭률이 높은 기사를 미리 저장
- 모델링 처리 : 과거 행동에서 사용자 선호도 학습
- 스코어링 처리 : 학습 완료된 모델을 사용해 기사의 점수를 부여
- 마지막으로 추천 기사를 사용자에게 전송하는 처리를 푸시 전송 기능이 담당

#### 후보 기사 선정
- 추천 후보가 되는 기사를 선택
- N일 이전의 최신 기사만 후보 기사로 필터
- 과거 푸시 전송 이력이 있는 후보 및 불쾌한 기사 등을 제외
- ctr이 높은 순서로 전송 후보 기사 채택

#### 모델링
- 사용자, 기사 특징을 활용해 사용자가 푸시 기사를 열지 말지에 대한 예측 모델을 구축

#### 스코어링
- 학습 완료된 모델에 대해 사용자, 기사 특징량을 입력하고 각 사용자 각 후보 기사에 대한 점수를 계산한 후 이 점수를 DB에 저장
- 전송 시점까지 저장

#### 푸시 전송
- 푸시 전송 처리에서는 각 사용자에게 추천 기사를 전송

### 6.3.2 실시간 추천
![뉴스 서비스의 실시간 추천 과정](./images/img_6-14.png)

- 행동 로그나 기사 정보를 사용해 클릭률이 높은 기사를 미리 db에 저장
- 모델링 처리 : 사용자 과거 행동에서 사용자 선호도를 학습
- 스코어링과 재순위 처리 : 기사 특징량과 사용자 특징량의 유사도를 계산하고 사용자에게 기사를 전송
- 사용자가 기사를 클릭할 때마다 특징량을 업데이트

#### 스코어링과 재순위
- 사용자, 기사 피처를 db에서 읽은 후 요청 사용자에 대한 후보 기사의 점수를 산출
- 응답시간에 주의하여 처리 성능을 튜닝해야 함
- 스코어링은 각 아티클에서 독립적으로 계산하므로 병렬로 계산 가능
- 응답 시간을 줄이지 못한다면 근사 최근접 탐색을 고려
- 캐시 도입도 응답 시간을 줄이는데 도움을 줌
- 재순위처리는 사용자가 과거 명시적으로 수행한 피드백과 입고 시간에 대한 기사 최신성을 고려해 나열

#### 특징량 업데이트
- 뉴스 추천은 사용자의 흥미나 관심이 단기간에 크게 변하기도 하므로 사용자 행동이 나타날 떄 그 결과를 사용자 흥미로 반영하는 것이 좋음
- 특징량 업데이트를 위한 입력은 사용자가 과거에 클릭한 기사의 특징량과 새로 클릭한 기사의 특징량
- 이 입력들을 학습 완료 모델을 통해 사용자의 특성으로 치환하고 DB에 저장

![특징량 업데이트](./images/img_6-15.png)